In [ ]:

import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import sklearn 
import numpy as np
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder  
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

!pip install transformers
!pip install torch
!pip install nltk

filepath=r'C:\Users\Admin\OneDrive\Desktop\dsai-nlp\complaints.csv'
f1=r'C:\Users\Admin\OneDrive\Desktop\dsai-nlp\final_dataframe.csv'
df=pd.read_csv(filepath)
df2=pd.read_csv(f1)

#Handling missing values
impute=SimpleImputer(strategy='most_frequent')
df=pd.DataFrame(impute.fit_transform(df),columns=df.columns)
                     

#data preparation
x=df['narrative'].values
unique=df["product"].unique()
unique
df_encoded = pd.get_dummies(df['product'])
df = pd.concat([df, df_encoded], axis=1)

y = np.argmax(df[['credit_card', 'credit_reporting', 'debt_collection', 'mortgages_and_loans', 'retail_banking']].values, axis=1)



#data splitting
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x_train_vectorized = vectorizer.fit_transform(x_train)

#train the RfR
model=RandomForestClassifier()
model.fit(x_train_vectorized,y_train)

#Model evaluation
#making predictions

y_pred=model.predict(x_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

#calculate metrics
accuracy=accuracy_score(y_test,y_pred,average='weighted')
precision=precision_score(y_test,y_pred,average='weighted')
recall=recall_score(y_test,y_pred,average='weighted')
f1=f1_score(y_test,y_pred,average='weighted')
report=classification_report(y_test,y_pred,average='weighted')

print(accuracy)
print(precision)
print(f1)
print(recall)
print(report)


#BERT
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

seq_len = [len(i.split()) for i in x_train]
#inputs for BERT
inputs = tokenizer(X.tolist(), return_tensors='pt', padding=True, truncation=True, max_length=128)
attention_mask = inputs['attention_mask']
#label-->tensor
labels_tensor = torch.tensor(y.tolist())

#forward pass
outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels_tensor)
loss = outputs.loss  # Loss for backpropagation
logits = outputs.logits  # Raw predictions

inputs_test = tokenizer(x_test.tolist(), return_tensors='pt', padding=True, truncation=True, max_length=128)
with torch.no_grad():
    outputs_test = bert_model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    predictions = torch.argmax(outputs_test.logits, dim=-1)

# Convert predictions to numpy array for sklearn metrics
predictions = predictions.numpy()

# Evaluate BERT Model
print("\nBERT Classifier Metrics:")
print("Accuracy:", accuracy_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions, average='weighted'))
print("Recall:", recall_score(y_test, predictions, average='weighted'))
print("F1 Score:", f1_score(y_test, predictions, average='weighted'))
print(classification_report(y_test, predictions, target_names=label_encoder.classes_))


#FINE TUNING
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Define the parameter distributions for RandomizedSearchCV
param_dist = {
    'countvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'multinomialnb__alpha': uniform(0.1, 1.0),
}

model = make_pipeline(CountVectorizer(), MultinomialNB())
randomized_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
# Train the model with the best hyperparameters
randomized_search.fit(X_train, y_train)
# Get the best model from the randomized search
best_model = randomized_search.best_estimator_
# Predict on the test set
y_pred_best = best_model.predict(X_test)
# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred_best)
print('Best Model Accuracy',accuracy)

print("Best Hyperparameters:", randomized_search.best_params_)

print("Classification Report:\n", classification_report(y_test, y_pred_best))



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
